In [32]:
!python -V

Python 3.9.21


In [33]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [34]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [45]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/songlisakim/workspace/mlops-learning/zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1748165225523, experiment_id='1', last_update_time=1748165225523, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [46]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [56]:
train_data_path = './data/green_tripdata_2021-01.parquet'
val_data_path = './data/green_tripdata_2021-02.parquet'
df_train = read_dataframe(train_data_path)
df_val = read_dataframe(val_data_path)

In [48]:
len(df_train), len(df_val)

(73908, 61921)

In [49]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [50]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [51]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [52]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715209766079

In [53]:
import os

def ensure_directory_exists(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        print(f"Directory created: {directory_path}")
    else:
        print(f"Directory already exists: {directory_path}")


In [54]:
model_dir='models'
ensure_directory_exists(model_dir)

with open(f'{model_dir}/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

Directory already exists: models


In [58]:
with mlflow.start_run():
    # Parameters
    alpha = 0.01
    
    # Training
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    # Inference
    y_pred = lr.predict(X_val)
    
    # Evaluation
    rmse = root_mean_squared_error(y_val, y_pred)
    
    mlflow.set_tag("developer", "songeun")
    mlflow.log_param("train-data-path", train_data_path)
    mlflow.log_param("valid-data-path", val_data_path)
    mlflow.log_param("alpha", alpha)
    mlflow.log_metric("rmse", rmse)



In [ ]:
import xgboost as xgb

# hyperopt is a library which uses bbayesian methods to find best hyperparam.
# fmin: to minimize output cost
# tpe: algorithm
# hp: contains different methods to define search space
# STATUS_ok: signal to send at the end of each run
# Trials: keep track of info from each run
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope


In [67]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)


In [71]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
    return {'loss': rmse, 'status': STATUS_OK}

In [72]:
# the ranges we want hyperop to explore hyper parameters
search_space =  { # xgboost hyperparameters
                                                             # uniform(low, high): return value uniformly btw low and high 
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)), # round(uniform(low, high) /q) * q
    'learning_rate': hp.loguniform('learning_rate', -3, 0), # exp(uniform(low, high)) -> 0.05~1
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42,
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials())





[0]	validation-rmse:7.92028                           
[1]	validation-rmse:6.97724                           
[2]	validation-rmse:6.76179                           
[3]	validation-rmse:6.70883                           
[4]	validation-rmse:6.68555                           
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/opt/homebrew/anaconda3/envs/zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:09:11] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[5]	validation-rmse:6.67810                           
[6]	validation-rmse:6.67596                           
[7]	validation-rmse:6.67378                           
[8]	validation-rmse:6.66963                           
[9]	validation-rmse:6.66333                           
[10]	validation-rmse:6.65765                          
[11]	validation-rmse:6.65187                          
[12]	validation-rmse:6.64508                          
[13]	validation-rmse:6.64292                          
[14]	validation-rmse:6.63848                          
[15]	validation-rmse:6.63340                          
[16]	validation-rmse:6.63136                          
[17]	validation-rmse:6.62702                          
[18]	validation-rmse:6.62335                          
[19]	validation-rmse:6.62121                          
[20]	validation-rmse:6.61885                          
[21]	validation-rmse:6.61533                          
[22]	validation-rmse:6.61237                          
[23]	valid

/opt/homebrew/anaconda3/envs/zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:09:24] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.33736                                                   
[1]	validation-rmse:10.58676                                                   
[2]	validation-rmse:9.94540                                                    
[3]	validation-rmse:9.40294                                                    
[4]	validation-rmse:8.94385                                                    
[5]	validation-rmse:8.56098                                                    
[6]	validation-rmse:8.22759                                                    
[7]	validation-rmse:7.95654                                                    
[8]	validation-rmse:7.72322                                                    
[9]	validation-rmse:7.53927                                                    
[10]	validation-rmse:7.38249                                                   
[11]	validation-rmse:7.24779                                                   
[12]	validation-rmse:7.13977            

...

[0]	validation-rmse:11.31251                                                    
[1]	validation-rmse:10.54513                                                    
[2]	validation-rmse:9.89517                                                     
[3]	validation-rmse:9.34333                                                     
[4]	validation-rmse:8.88282                                                     
[5]	validation-rmse:8.49748                                                     
[6]	validation-rmse:8.17203                                                     
[7]	validation-rmse:7.90437                                                     
[8]	validation-rmse:7.67975                                                     
[9]	validation-rmse:7.49643                                                     
[10]	validation-rmse:7.34626                                                    
[11]	validation-rmse:7.22105                                                    
[12]	validation-rmse:7.11396

/opt/homebrew/anaconda3/envs/zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:34:37] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.76982                                                    
[1]	validation-rmse:11.35796                                                    
[2]	validation-rmse:10.97575                                                    
[3]	validation-rmse:10.62135                                                    
[4]	validation-rmse:10.29324                                                    
[5]	validation-rmse:9.98960                                                     
[6]	validation-rmse:9.70919                                                     
[7]	validation-rmse:9.45051                                                     
[8]	validation-rmse:9.21201                                                     
[9]	validation-rmse:8.99229                                                     
[10]	validation-rmse:8.79011                                                    
[11]	validation-rmse:8.60491                                                    
[12]	validation-rmse:8.43482

/opt/homebrew/anaconda3/envs/zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:35:30] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.52039                                                    
[1]	validation-rmse:10.90651                                                    
[2]	validation-rmse:10.36258                                                    
[3]	validation-rmse:9.88090                                                     
[4]	validation-rmse:9.45884                                                     
[5]	validation-rmse:9.08810                                                     
[6]	validation-rmse:8.76441                                                     
[7]	validation-rmse:8.48289                                                     
[8]	validation-rmse:8.23425                                                     
[9]	validation-rmse:8.02192                                                     
[10]	validation-rmse:7.83547                                                    
[11]	validation-rmse:7.67610                                                    
[12]	validation-rmse:7.53746

In [61]:
from math import exp
exp(-3)

0.049787068367863944

In [ ]:
params = {
    'learning_rate': 0.08689277546104228,
    'max_depth': 41,
    'min_child_weight': 1.0573384280854001,
    'objective': 'reg:linear',
    'reg_alpha': 0.019777208733018477,
    'reg_lambda': 0.03147085383119916,
    'seed': 42
}

mlflow.xgboost.autolog()
booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, "validation")],
    early_stopping_rounds=50
)
